In [ ]:
!pip install jiwer
!pip install bnunicodenormalizer
!pip install image-classifiers

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from jiwer import wer
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

import warnings
import random
import matplotlib.pyplot as plt
import json

from tqdm.auto import tqdm
from pandarallel import pandarallel
from IPython.display import display,Audio
from bnunicodenormalizer import Normalizer 
from pprint import pprint
from multiprocessing import Process
from classification_models.tfkeras import Classifiers

pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()
warnings.filterwarnings('ignore')
bnorm=Normalizer()

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
strategy = tf.distribute.get_strategy()

In [ ]:
errors=["common_voice_bn_31727562",
        'common_voice_bn_30998934',
        'common_voice_bn_31595526',
        'common_voice_bn_31534853',
        'common_voice_bn_31518061',
        'common_voice_bn_31518373',
        'common_voice_bn_31613621',
        'common_voice_bn_31555333',
        'common_voice_bn_31772113',
        'common_voice_bn_31605391',
        'common_voice_bn_31631175',
        'common_voice_bn_31563901',
        'common_voice_bn_31691690',
        'common_voice_bn_31692010',
        'common_voice_bn_31683653',
        'common_voice_bn_31692182',
        'common_voice_bn_31519976',
        'common_voice_bn_31675793',
        'common_voice_bn_31019914',
        'common_voice_bn_31660287',
        'common_voice_bn_31660384',
        'common_voice_bn_31557261',
        'common_voice_bn_31633101',
        'common_voice_bn_31599243',
        'common_voice_bn_31521515',
        'common_voice_bn_31777802',
        'common_voice_bn_31777848',
        'common_voice_bn_31669646',
        'common_voice_bn_31566083',
        'common_voice_bn_31530331',
        'common_voice_bn_31727697',
        'common_voice_bn_31513270',
        'common_voice_bn_31686295',
        'common_voice_bn_31753693',
        'common_voice_bn_31686334',
        'common_voice_bn_31765546',
        'common_voice_bn_31765548',
        'common_voice_bn_31662742',
        'common_voice_bn_31704856',
        'common_voice_bn_31635344',
        'common_voice_bn_31618327',
        'common_voice_bn_31743074',
        'common_voice_bn_31678862',
        'common_voice_bn_31626674',
        'common_voice_bn_31626677',
        'common_voice_bn_31523889',
        'common_voice_bn_31610804',
        'common_voice_bn_31769538',
        'common_voice_bn_31533273',
        'common_voice_bn_31445621',
        'common_voice_bn_31620650']
#---------------
# data filtering
#---------------
def filter_votes(x):
    p=x["path"]
    # avoid error data
    for pe in errors:
        if pe in p:
            return None
    # now process votes
    up=x["up_votes"]
    down=x["down_votes"]
    if up-down<=0:
        return "unv"
    elif up==0:
        return "unv"
    else:
        return up
# ------------------------- train data----------------------------------------
train_path="../input/train-wavs-voted-dl-sprint/train_files_wav"
df=pd.read_csv("../input/dlsprint/train.csv")
df["path"]=df["path"].progress_apply(lambda x:os.path.join(train_path,x).replace(".mp3",".wav"))
print("Total Data before filtering:",len(df))
df["up_votes"]=df.progress_apply(lambda x:filter_votes(x),axis=1)
df.dropna(subset=["up_votes"],inplace=True)
train=df.loc[df.up_votes!="unv"]
train.reset_index(drop=True,inplace=True)
print("Total Data after filtering:",len(train))
unv_df=df.loc[df.up_votes=="unv"]
unv_df.reset_index(drop=True,inplace=True)
print("Total unverified Data after filtering:",len(unv_df))
train=train[["path","sentence"]]
unv_df=unv_df[["path","sentence"]]
# ------------------------- eval data----------------------------------------
val_path="../input/validation-fileswav-format/validation_files_wav"
valid=pd.read_csv("../input/dlsprint/validation.csv")
valid=valid[["path","sentence"]]
valid["path"]=valid["path"].progress_apply(lambda x:os.path.join(val_path,x).replace(".mp3",".wav"))
print("Total validation Data :",len(valid))

In [ ]:
train.head()

In [ ]:
characters = [x for x in ['।', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঋ', 'এ', 'ঐ', 'ও', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯',' ']]
# Mapping characters to integers
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")
# Mapping integers back to original characters
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

In [ ]:
# An integer scalar Tensor. The window length in samples.
frame_length = 200
# An integer scalar Tensor. The number of samples to step.
frame_step = 80
# An integer scalar Tensor. The size of the FFT to apply.
# If not provided, uses the smallest power of 2 enclosing frame_length.
fft_length = 256


def encode_single_sample(wav_file, label):
    ###########################################
    ##  Process the Audio
    ##########################################
    # 1. Read wav file
    file = tf.io.read_file(wavs_path + wav_file)
    # 2. Decode the wav file
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    # 3. Change type to float
    audio = tf.cast(audio, tf.float32)
    # 4. Get the spectrogram
    spectrogram = tf.signal.stft(
        audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length
    )
    # 5. We only need the magnitude, which can be derived by applying tf.abs
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)
    # 6. normalisation
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)
    ###########################################
    ##  Process the label
    ##########################################
    # 7. Convert label to Lower case
    #label = tf.strings.lower(label)
    # 8. Split the label
    label = tf.strings.unicode_split(label, input_encoding="UTF-8")
    # 9. Map the characters in label to numbers
    label = char_to_num(label)
    # 10. Return a dict as our model is expecting two inputs
    return spectrogram,label

In [ ]:
import os
filenames = os.listdir("../input/train-wavs-voted-dl-sprint/train_files_wav")
from tqdm import tqdm

df = train.iloc[:len(filenames)]
to_take = []
label = []

for i in tqdm(range(len(train))):
    if train.path.iloc[i] in filenames:
        to_take.append(train.path.iloc[i])
        label.append(train.sentence.iloc[i])
df.path = to_take
df.sentence = label

train = df
print(train.shape)
train.head()

In [ ]:
wavs_path = "../input/train-wavs-voted-dl-sprint/train_files_wav/"

batch_size = 32
# Define the trainig dataset
train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(train["path"]), list(train["sentence"]))
)
train_dataset = (
    train_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

wavs_path = "../input/validation-fileswav-format/validation_files_wav/"
# Define the validation dataset
validation_dataset = tf.data.Dataset.from_tensor_slices(
    (list(valid["path"]), list(valid["sentence"]))
)
validation_dataset = (
    validation_dataset.map((encode_single_sample), num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)


In [ ]:
wavs_path = "../input/train-wavs-voted-dl-sprint/train_files_wav/"

fig = plt.figure(figsize=(8, 5))
for batch in train_dataset.take(1):
    spectrogram = batch[0][0].numpy()
    spectrogram = np.array([np.trim_zeros(x) for x in np.transpose(spectrogram)])
    label = batch[1][0]
    # Spectrogram
    label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
    print(label)
    ax = plt.subplot(2, 1, 1)
    ax.imshow(spectrogram, vmax=1)
    ax.set_title(label)
    ax.axis("off")
    # Wav
    file = tf.io.read_file(wavs_path + list(train["path"])[0])
    audio, _ = tf.audio.decode_wav(file)
    print(_)
    audio = audio.numpy()
    ax = plt.subplot(2, 1, 2)
    plt.plot(audio)
    ax.set_title("Signal Wave")
    ax.set_xlim(0, len(audio))
    display.display(display.Audio(np.transpose(audio), rate=16000))
plt.show()

In [ ]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    print(input_length,label_length)
    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [ ]:

import torch
import torch.nn as nn

class DotAttention(tf.keras.layers.Layer):
   
    def __init__(self):
        super().__init__()
        self.inf_val=-1e9
        
    def call(self,q, k, v, mask):
        
        matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
       
        # scale matmul_qk
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

        # add the mask to the scaled tensor.
        if mask is not None:
            scaled_attention_logits += (mask * self.inf_val)

        # softmax is normalized on the last axis (seq_len_k) so that the scores
        # add up to 1.
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

        output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

        return output
    
#--------------------------------------------------------------------

class PositionalEncoding(tf.keras.layers.Layer):
    '''
    tensorflow wrapper for positional encoding layer
    args:
      num_seq  :   incoming sequence length
      projection_dim  :   projection_dim
      use_torch_weights : torch weights help converge faster for basic dot attention
    '''
    def __init__(self,num_seq,projection_dim,use_torch_weights=False):
        super(PositionalEncoding, self).__init__()
        self.use_torch_weights=use_torch_weights
        self.projection_dim=projection_dim
        self.num_seq = num_seq
        self.projection = tf.keras.layers.Dense(units=projection_dim)
        if use_torch_weights:
            pos_emb              = nn.Embedding(num_seq+1,projection_dim)
            pos_emb_weight       = pos_emb.weight.data.numpy()
            self.position_embedding = tf.keras.layers.Embedding(input_dim=num_seq+1, output_dim=projection_dim,weights=[pos_emb_weight])
        else:
            
            self.position_embedding = tf.keras.layers.Embedding(input_dim=num_seq, output_dim=projection_dim)

    def call(self, x):
        positions = tf.range(start=0, limit=self.num_seq, delta=1)
        if x is None:
            return self.position_embedding(positions)
        encoded = self.projection(x) + self.position_embedding(positions)
        return encoded
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({'num_seq': self.num_seq,'projection_dim':self.projection_dim,"use_torch_weights":use_torch_weights})
        return config

#--------------------------------------------------------------------
 
class TransformerBlock(tf.keras.layers.Layer):
    '''
        transformer encoder block based on multihead self-attention
    '''
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.embed_dim=embed_dim
        self.num_heads=num_heads
        self.ff_dim   =ff_dim

        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    def get_config(self):
        config = super().get_config().copy()
        config.update({'embed_dim': self.embed_dim,
                       'num_heads': self.num_heads,
                       'ff_dim':self.ff_dim})
        return config

In [ ]:
def attend(x,num_heads,num_blocks,reshape=True):
    '''
        basic self-attention wrapper
        args:
            x : input tensor
            num_heads : heads to use in multihead attention
            num_blocks: how many attention blocks to use 
    '''
    bs,h,w,nc=x.shape
    x = tf.keras.layers.Reshape((h*w,nc))(x)
    x = PositionalEncoding(h*w,nc)(x)
    for _ in range(num_blocks):
        x=TransformerBlock(embed_dim=nc, num_heads=num_heads,ff_dim=4*nc)(x)
    if reshape:
        x = tf.keras.layers.Reshape((h,w,nc))(x)
    return x


def create_basic_model(cfg):
    '''
        creates a basic cnn-seftattention-positional-attention based model
        **flow**
        cnn_feat(input_image)---> h//f,w//f,c shaped tensor = feat
        attend(feat)---> self attention applied on the features= enc
        pos_attention(enc)--> align encoded features with positional data=logits
    '''
    #-----------cnn feature extractor------------------
    cnn,_ = Classifiers.get(cfg.backbone)
    cnn = cnn(cfg.img_dim,weights=None,include_top=False)
    inp = cnn.input
    x   = cnn.output
    bs,h,w,fc=x.shape
    if fc!=cfg.embed_dim:
        x=tf.keras.layers.Conv2D(cfg.embed_dim,3,padding='same')(x)
    print("model input:",inp.shape)
    print("cnn feat:",x.shape)
    #-----------self attention------------------
    x=attend(x,cfg.num_blocks,cfg.num_heads,reshape=False)
    print("feat attention (seq,embed_dim):",x.shape)
    #-----------positional attention------------------
    pos=PositionalEncoding(cfg.pos_max,cfg.embed_dim,use_torch_weights=True)(None)
    attn=DotAttention()(pos,x,x,None)
    print("positional attention (pos_max,embed_dim):",attn.shape)
    x=tf.keras.layers.Dense(cfg.logits_len)(attn)
    print("logits(pos_max,logits_len):",x.shape)
    model = tf.keras.Model(inputs=inp,outputs=x)
    return model

In [ ]:
class cfg:
    backbone='resnet18'
    img_dim =(64,1088,1)     #features.shape
    pos_max =200             #np.array(encode_label(sen)).shape
    num_heads=8              # number of self-attention heads
    num_blocks=4             # number of transformer blocks
    embed_dim =256           # reduced channel for sequencing
    logits_len =85   

In [ ]:
with strategy.scope():
    model=create_basic_model(cfg)
model.summary()

In [ ]:
#------------------metrics---------------------------
def C_acc(y_true, y_pred):
    '''
        calculates how many charecters are predicted correctly
    '''
    pad_value=0 # label pad index
    accuracies = tf.equal(tf.cast(y_true,tf.int64), tf.argmax(y_pred, axis=2))
    mask = tf.math.logical_not(tf.math.equal(y_true,pad_value))
    accuracies = tf.math.logical_and(mask, accuracies)
    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)
#------------------loss--------------------------
class CharLoss(tf.keras.losses.Loss):
    """
        a loss function to estimate charecter accuracy loss ignoring pad value
    """
    def __init__(self,pad_value):
        super(CharLoss, self).__init__(name="char_loss")
        self.pad_value=pad_value
        self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
    def call(self, y_true, y_pred):
        mask = tf.math.logical_not(tf.math.equal(y_true, self.pad_value))
        loss_ = self.loss_object(y_true, y_pred)
        mask = tf.cast(mask, dtype=loss_.dtype)
        loss_ *= mask
        return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

class CTCLoss(tf.keras.losses.Loss):
    """ A class that wraps the function of tf.nn.ctc_loss. 
    
    Attributes:
        logits_time_major: If False (default) , shape is [batch, time, logits], 
            If True, logits is shaped [time, batch, logits]. 
        blank_index: Set the class index to use for the blank label. default is
            -1 (num_classes - 1). 
    """

    def __init__(self, logits_time_major=False, name='ctc_loss'):
        super().__init__(name=name)
        self.logits_time_major = logits_time_major

    def call(self, y_true, y_pred):
        """ 
            Computes CTC (Connectionist Temporal Classification) loss. 
        """
        y_true = tf.cast(y_true, tf.int32)
        logit_length = tf.fill([tf.shape(y_pred)[0]], tf.shape(y_pred)[1])
        label_length = tf.fill([tf.shape(y_true)[0]], tf.shape(y_true)[1])
        loss = tf.nn.ctc_loss(
            labels=y_true,
            logits=y_pred,
            label_length=label_length,
            logit_length=logit_length,
            logits_time_major=self.logits_time_major,
            blank_index=0)
        return tf.math.reduce_mean(loss)

In [ ]:
# early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, 
                                                  verbose=1, 
                                                  mode = 'auto') 
callbacks = [tf.keras.callbacks.ModelCheckpoint("model.h5",
                                                save_best_only=True,
                                                save_weights_only=True,
                                                verbose=1),
             early_stopping]

In [ ]:
with strategy.scope():

    lr_schedule = tf.keras.experimental.CosineDecay(initial_learning_rate=0.0001,
                                                             decay_steps=600000,
                                                             alpha= 0.01)

    model.compile(optimizer=tf.keras.optimizers.Adam(lr_schedule),
                  loss=CharLoss(char_to_num.get_vocabulary()),
                  metrics=[C_acc])

In [ ]:
# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]
    # Iterate over the results and get back the text
    output_text = []
    for result in results:
        result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
        output_text.append(result)
    return output_text


# A callback class to output a few transcriptions during training
class CallbackEval(keras.callbacks.Callback):
    """Displays a batch of outputs after every epoch."""

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def on_epoch_end(self, epoch: int, logs=None):
        predictions = []
        targets = []
        for batch in self.dataset:
            X, y = batch
            batch_predictions = model.predict(X)
            batch_predictions = decode_batch_predictions(batch_predictions)
            predictions.extend(batch_predictions)
            for label in y:
                label = (
                    tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
                )
                targets.append(label)
        wer_score = wer(targets, predictions)
        print("-" * 100)
        print(f"Word Error Rate: {wer_score:.4f}")
        print("-" * 100)
        for i in np.random.randint(0, len(predictions), 2):
            print(f"Target    : {targets[i]}")
            print(f"Prediction: {predictions[i]}")
            print("-" * 100)

In [ ]:
EPOCHS=1
BATCH_SIZE = 8
STEPS_PER_EPOCH=len(train_dataset)//BATCH_SIZE
EVAL_STEPS=len(validation_dataset)//BATCH_SIZE


history=model.fit(train_dataset,
                  epochs=EPOCHS,
                  steps_per_epoch=STEPS_PER_EPOCH,
                  verbose=1,
                  validation_data=validation_dataset,
                  validation_steps=EVAL_STEPS, 
                  callbacks=callbacks)

In [ ]:
predictions = []
targets = []
i=0
for batch in tqdm(validation_dataset):
    X, y = batch
    batch_predictions = model.predict(X)
    batch_predictions = decode_batch_predictions(batch_predictions)
    print(batch_predictions)
    i+=1
    if i>5:
        break
    predictions.extend(batch_predictions)
    for label in y:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        targets.append(label)
wer_score = wer(targets, predictions)
print("-" * 100)
print(f"Word Error Rate: {wer_score:.4f}")
print("-" * 100)
for i in np.random.randint(0, len(predictions), 5):
    print(f"Target    : {targets[i]}")
    print(f"Prediction: {predictions[i]}")
    print("-" * 100)

# Inference

In [ ]:
test = pd.read_csv("../input/dlsprint/sample_submission.csv")

In [ ]:
test.path = test.path.apply(lambda x:x.replace(".mp3",".wav"))
test.head()

In [ ]:
wavs_path = "../input/test-wav-files-dl-sprint/test_files_wav/"

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices(
    (list(test["path"]), list(test["sentence"]))
)
test_dataset = (
    test_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
    .padded_batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

In [ ]:
predictions = []
targets = []
for batch in tqdm(validation_dataset):
    X, y = batch
    batch_predictions = model.predict(X)
    batch_predictions = decode_batch_predictions(batch_predictions)
    predictions.extend(batch_predictions)
    for label in y:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
        targets.append(label)
wer_score = wer(targets, predictions)
print("-" * 100)
print(f"Word Error Rate: {wer_score:.4f}")
print("-" * 100)
for i in np.random.randint(0, len(predictions), 5):
    print(f"Target    : {targets[i]}")
    print(f"Prediction: {predictions[i]}")

In [ ]:
sub = pd.read_csv("../input/dlsprint/sample_submission.csv")

In [ ]:
sub.sentence = predictions
sub.head()

In [ ]:
sub.to_csv("submission_ctc_tutorial",index=False)